# Statistical Features

In [1]:
#import necessary libraries
from mido import MidiFile, MidiTrack, Message
from mido.midifiles.meta import MetaMessage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import mido
import os
import math
import numpy as np
import pandas as pd
import statistics
from scipy.stats import entropy
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

<h3>Data Extraction

In [2]:
#read midis for jigs
os.getcwd()
files_j = []
os.chdir("/home/girija/Documents/Project/midifiles")
for root, dirs, files in os.walk("."):
    for file in files:
        files_j.append(mido.MidiFile(file))
        
#read midis for reels
os.getcwd()
files_r = []
os.chdir("/home/girija/Documents/Project/reels")
for root, dirs, files in os.walk("."):
    for file in files:
        files_r.append(mido.MidiFile(file))

In [3]:
#get pitch and duration for jigs
pitch_j = []
velocity_j = []
for mid in files_j:
    p = []
    v = []
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                if(message.type == 'note_off'):
                    p.append(message.note)
                    v.append(message.velocity)
    pitch_j.append(p)
    velocity_j.append(v)
    
#get pitch and duration for reels
pitch_r = []
velocity_r = []
for mid in files_r:
    p = []
    v = []
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                if(message.type == 'note_off'):
                    p.append(message.note)
                    v.append(message.velocity)  
    pitch_r.append(p)
    velocity_r.append(v)

<h3>Histogram of Pitch Values

In [4]:
#histogram of pitch for jigs
hist_j = []
for tune in pitch_j:
    hist = list(np.zeros(12))
    for pitch in tune:
        tmp = pitch%12
        hist[tmp]+=1
    hist_j.append(hist)
    
#histogram of pitch for reels
hist_r = []
for tune in pitch_r:
    hist = list(np.zeros(12))
    for pitch in tune:
        tmp = pitch%12
        hist[tmp]+=1
    hist_r.append(hist)

In [5]:
#histogram of pitch
#statistical features for jigs
avg_j = []
stddev_j = []
entropy_j = []
for item in hist_j:
    avg_j.append(statistics.mean(item))
    stddev_j.append(statistics.stdev(item))
    entropy_j.append(entropy(item))

#statistical features for reels
avg_r = []
stddev_r = []
entropy_r = []
for item in hist_r:
    avg_r.append(statistics.mean(item))
    stddev_r.append(statistics.stdev(item))
    entropy_r.append(entropy(item))
    
l1 = list(np.zeros(100))
l2 = list(np.ones(100))

<h3>Dataset Creation

In [6]:
averages = avg_j+avg_r
sds = stddev_j+stddev_r
entropies = entropy_j+entropy_r

In [7]:
dataset = []
for i in range(100):
    tmp = []
    tmp.append(averages[i])
    tmp.append(sds[i])
    tmp.append(entropies[i])
    dataset.append(tmp)

<h3>K-Means Clustering

In [8]:
def KMeansClustering(dataset, k, max_itr):
    #create an empty list of centroids
    centroids = []
    #select first 'k' datapoints as the centroids
    for i in range(k):
        centroids.append(dataset[i])
    
    for itr in range(max_itr):
        clusters = []
        #find distance between a data-point and each centroid
        for point in dataset:
            distances = [np.linalg.norm(np.asarray(point)-np.asarray(centroid)) for centroid in centroids]
            #select centroid at minimum distance
            clusters.append(distances.index(min(distances)))

        #empty dictionary to store data-points as per cluster numbers    
        data = {}
        for i in range(k):
            data[i] = []
        
        #append data-points to the list based on cluster number    
        for i in range(len(dataset)):
            data[clusters[i]].append(dataset[i])
        
        #update centroids based on mean of data-points in the cluster    
        centroids = []
        for point in data:
            centroids.append(np.mean(data[point],axis = 0))
            
    return(clusters)

<h3>Silhouette Analysis

In [9]:
#Evaluation using silhouette analysis
def silhouette_analysis(datasets,k, clusters):
    a = []
    intra_cluster = []
    for i in range(k):
        tmp = []
        #find tunes belonging to a particular cluster
        for j in range(len(datasets)):
            if(i==clusters[j]):
                tmp.append(j)
        intra_cluster.append(tmp)
    
    #find intra-cluster distances of all data-points
    for cluster in intra_cluster:
        for i in range(len(cluster)):
            other_points = cluster[:i]+cluster[i+1:]
            dist = []
            flag = 0
            for point in other_points:
                if(len(other_points)>1):
                    dist.append(np.linalg.norm(np.asarray(datasets[point])-np.asarray(datasets[cluster[i]])))
                else:
                    dist.append(np.mean(datasets[point]))
            a.append(np.mean(dist))
    
    #find inter-cluster distances of all data-points
    inter_clusters = []
    for i in range(len(intra_cluster)):
        inter_clusters.append(list(itertools.chain.from_iterable(intra_cluster[:i]+intra_cluster[i+1:])))
    
    b = []
    for i in range(len(intra_cluster)):
        for j in range(len(inter_clusters)):
            if(i==j):
                for given_point in intra_cluster[i]:
                    dist = []
                    for other_point in inter_clusters[j]:
                        dist.append(np.linalg.norm(np.asarray(datasets[given_point])-np.asarray(datasets[other_point])))
                    b.append(min(dist))
    
    #find silhouette co-efficient of each point
    result = []
    for i in range(len(a)):
        for j in range(len(b)):
            if(i==j):
                res = (b[i]-a[i])/(max(a[i],b[i]))
                if(not math.isnan(res)):
                    result.append(res)
    return(statistics.mean(result))

<h3>Result of Histogram of Pitch on K-Means

In [10]:
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 2, 100)
    total+=silhouette_analysis(dataset,2, clusters)

In [11]:
print("Silhouette co-efficient for Histogram of Pitch:",total)

Silhouette co-efficient for Histogram of Pitch: 0.3666897375820209


<h3>Histogram of Velocities

In [12]:
#histogram of velocities for jigs
hist_j_v = []
for tune in velocity_j:
    hist = list(np.zeros(12))
    for pitch in tune:
        tmp = pitch%12
        hist[tmp]+=1
    hist_j_v.append(hist)
    
#histogram of velocities for reels
hist_r_v = []
for tune in velocity_r:
    hist = list(np.zeros(12))
    for pitch in tune:
        tmp = pitch%12
        hist[tmp]+=1
    hist_r_v.append(hist)

In [13]:
#histogram of velocities
#statistical features for jigs
avg_j_v = []
stddev_j_v = []
entropy_j_v = []
for item in hist_j_v:
    avg_j_v.append(statistics.mean(item))
    stddev_j_v.append(statistics.stdev(item))
    entropy_j_v.append(entropy(item))

#statistical features for reels
avg_r_v = []
stddev_r_v = []
entropy_r_v = []
for item in hist_r_v:
    avg_r_v.append(statistics.mean(item))
    stddev_r_v.append(statistics.stdev(item))
    entropy_r_v.append(entropy(item))

In [14]:
#for dataset creation
averages = avg_j_v+avg_r_v
sds = stddev_j_v+stddev_r_v
entropies = entropy_j_v+entropy_r_v

In [15]:
#create dataset
dataset = []
for i in range(100):
    tmp = []
    tmp.append(averages[i])
    tmp.append(sds[i])
    tmp.append(entropies[i])
    dataset.append(tmp)

<h3>Result of Histogram of Velocities on K-Means

In [16]:
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 2, 100)
    total+=silhouette_analysis(dataset,2, clusters)

In [17]:
print("Silhouette co-efficient for Histogram of Velocities:",total)

Silhouette co-efficient for Histogram of Velocities: 0.36095864686109763
